In [1]:
import pandas as pd    #importing required libraries
import sqlite3

In [4]:
orders_df = pd.read_csv("orders.csv")
orders_df.head()      #starting 5 orders details

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users_df=pd.read_json("users.json")
users_df.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
con = sqlite3.connect(":memory:")
cursor = con.cursor()
with open("restaurants.sql","r") as f:
  sql_script = f.read()
cursor.executescript(sql_script)

restaurants_df = pd.read_sql(
    "SELECT * FROM restaurants",
    con
)

restaurants_df.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [9]:
orders_users_df = orders_df.merge(
    users_df,
    on="user_id",
    how="left"       #merging of oders
)

In [10]:
final_df = orders_users_df.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"        #merging orders and restaurants

)

In [11]:
final_df.shape,final_df.isnull().sum()   # check about NULL values

((10000, 12),
 order_id             0
 user_id              0
 restaurant_id        0
 order_date           0
 total_amount         0
 restaurant_name_x    0
 name                 0
 city                 0
 membership           0
 restaurant_name_y    0
 cuisine              0
 rating               0
 dtype: int64)

In [16]:
final_df.to_csv ("final_food_delivery_dataset.csv",index=False)

df = pd.read_csv("final_food_delivery_dataset.csv")

df["order_date"] = pd.to_datetime(df["order_date"],dayfirst=True)

df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [17]:
df[df["membership"]=="Gold"]\
  .groupby("city")["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [19]:
df.groupby("cuisine")["total_amount"]\
.mean()\
.sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [25]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)



,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [23]:
user_total = df.groupby("user_id")["total_amount"].sum()

(user_total > 1000).sum()

np.int64(2544)

In [27]:
df["rating_range"]=pd.cut(
    df["rating"],
    bins=[3.0,3.5,4.0,4.5,5.0],
    labels =["3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"],
    include_lowest=True
    )

df.groupby("rating_range")["total_amount"]\
.sum()\
.sort_values(ascending=False)

/tmp/ipython-input-2993494981.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]\


,total_amount
rating_range,
4.6-5.0,2197030.75
3.0-3.5,2136772.70
4.1-4.5,1960326.26
3.6-4.0,1717494.41


In [28]:
df[df["membership"]=="Gold"]\
 .groupby("city")["total_amount"] \
 .mean()\
 .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [29]:
pd.DataFrame({
    "restaurant_count":df.groupby("cuisine")["restaurant_id"].nunique(),
    "total_revenue":df.groupby("cuisine")["total_amount"].sum()
  }).sort_values(
      ["restaurant_count","total_revenue"],
      ascending=[True,False]
  )

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [30]:
gold_orders = df[df["membership"]=="Gold"].shape[0]
total_orders = df.shape[0]

round((gold_orders/total_orders)*100)


50

In [31]:
df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount","mean"),
    total_orders=("order_id","count")
).query("total_orders<20")\
  .sort_values("avg_order_value",ascending=False)

,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [33]:
df.groupby(["membership","cuisine"])["total_amount"]\
.sum()\
.sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [37]:
df["quarter"] = df["order_date"].dt.quarter
df.groupby(["quarter"])["total_amount"]\
.sum()\
.sort_values(ascending=False)

,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [38]:
df[df["membership"]=="Gold"].shape[0]


4987

In [39]:
round(df[df["city"]=="Hyderabad"]["total_amount"].sum())

1889367

In [41]:
df["user_id"].nunique()

2883

In [42]:
round(
    df[df["membership"]=="Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [43]:
df[df["rating"]>=4.5].shape[0]

3374

In [45]:
top_gold_city=(
    df[df["membership"]=="Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city

df[(df["membership"]=="Gold")&
   (df["city"]==top_gold_city)].shape[0]


1337